In [1]:
import os

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1649603212487_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
%%bash
ls /tmp

1649603768116-0
1649613934157-0
7002467630638818348
7364206898285743039
blockmgr-1d2eda59-3add-45be-9f4e-a41c09f54364
blockmgr-59b841b8-d377-4944-abc7-476265f8fd97
blockmgr-8b6f17c6-e79d-4102-ab1b-89a0d5103bcc
hadoop-hdfs-namenode.pid
hadoop-state-pusher.config
hadoop-yarn-yarn
hsperfdata_emr-notebook
hsperfdata_hadoop
hsperfdata_hdfs
hsperfdata_kms
hsperfdata_livy
hsperfdata_mapred
hsperfdata_root
hsperfdata_spark
hsperfdata_yarn
jetty-172_31_17_183-8088-_-any-69188706602102375.dir
jetty-ip-172-31-17-183_us-east-2_compute_internal-19888-_-any-6943295675471287217.dir
jetty-ip-172-31-17-183_us-east-2_compute_internal-20888-_-any-8390878317201690973.dir
jetty-ip-172-31-17-183_us-east-2_compute_internal-8188-_-any-6445470360603456709.dir
jetty-ip-172-31-17-183_us-east-2_compute_internal-9870-hdfs-_-any-7253595740607422041.dir
libleveldbjni-64-1-8583846394274074828.8
liblz4-java-5417858812988783083.so
liblz4-java-5417858812988783083.so.lck
livyConf3440113674903466731.properties
livyConf700

In [67]:
hadoop = sc._jvm.org.apache.hadoop
hdfs = hadoop.fs.FileSystem.get(sc._jsc.hadoopConfiguration())

def hdfs_path(path_str):
    return hadoop.fs.Path(path_str)


def list_hdfs(path_str):
    # https://stackoverflow.com/a/45178292/11262633

    conf = hadoop.conf.Configuration() 
    path = hdfs_path(path_str)

    for f in hdfs.listStatus(path):
        print(f.getPath(), f.getLen())
        


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
list_hdfs('/tmp')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/date.txt 29
hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/entity-file-history 0
hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/hadoop-yarn 0
hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/test_hdfs.txt 0

In [61]:
#https://sparkbyexamples.com/spark/spark-rename-and-delete-file-directory-from-hdfs/#:~:text=In%20order%20to%20delete%20a,()%20method%20of%20Hadoop%20FileSystem.&text=delete()%20method%20of%20FileSystem,both%20File%20and%20a%20Directory.

targ_file_str = '/tmp/test2.txt'
targ_file = hdfs_path(targ_file_str)

if hdfs.exists(targ_file):
    result = hdfs.delete(targ_file, True)
    if result:
        print(f'File {targ_file_str} deleted')
    else:
        print(f'Error deleting file {targ_file_str}')
else:
    print(f'File {targ_file_str} not found')    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

File /tmp/test2.txt not found

In [70]:
# https://stackoverflow.com/a/58912480/11262633

raw_out = hdfs.create(targ_file)  # FSDataOutputStream
out_stream = sc._jvm.java.io.BufferedOutputStream(raw_out)

for f in hdfs.listStatus(hdfs_path('/tmp/')):
    in_path = f.getPath()
    if not str(in_path).endswith('.txt'):
        continue
    print(in_path, type(in_path))
    raw_in = hdfs.open(in_path)
    in_stream = sc._jvm.java.io.BufferedInputStream(raw_in)

    while in_stream.available() > 0:
        out_stream.write(in_stream.read()) 
        out_stream.flush()
    in_stream.close()
    break

out_stream.close()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/date.txt <class 'py4j.java_gateway.JavaObject'>

In [71]:
list_hdfs('/tmp')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/date.txt 29
hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/entity-file-history 0
hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/hadoop-yarn 0
hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/test2.txt 29
hdfs://ip-172-31-17-183.us-east-2.compute.internal:8020/tmp/test_hdfs.txt 0

In [4]:
#https://stackoverflow.com/a/65455869/11262633
from pyspark.sql.functions import base64, col
img_df = spark.read.format("image").load("s3://multimedia-commons/data/images/000/24a/00024a73d1a4c32fb29732d56a2.jpg")
proc_df = img_df.select(base64(col("image.data"))).alias('encoded')

#proc_df.coalesce(1).write.format("text").save('/tmp/sample2.jpg')  *** Creates a base64 text file

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o140.save.
: java.lang.ClassNotFoundException: Failed to find data source: binary. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:692)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:746)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:993)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:311)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:293)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java